In [2]:
import pyspark

In [6]:
sc = pyspark.SparkContext(appName="Matrix Multiplication")

In [7]:
matrice1 = sc.textFile("matricequedes1.txt")

In [9]:
matrice1.collect()

['0 0 1',
 '0 1 1',
 '0 2 1',
 '1 0 1',
 '1 1 1',
 '1 2 1',
 '2 0 1',
 '2 1 1',
 '2 2 1']